# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install required package

In [2]:
!pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.2 MB/s eta 0:00:00


# Import Libraries

In [3]:
import ast
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from Bio import SeqIO
import matplotlib.pyplot as plt

# Define file paths

In [4]:
# Paths to input data
fasta_path = "/content/drive/MyDrive/Eksperimen_CAFA5/Train/train_sequences.fasta"
terms_path = "/content/drive/MyDrive/Eksperimen_CAFA5/Train/train_terms.tsv"
taxonomy_path = "/content/drive/MyDrive/Eksperimen_CAFA5/Train/train_taxonomy.tsv"

# Read target FASTA data
fasta_target_path = "/content/drive/MyDrive/Eksperimen_CAFA5/Test (Targets)/testsuperset.fasta"

In [5]:
# Load data
fasta_data = pd.DataFrame([(record.id, str(record.seq)) for record in SeqIO.parse(fasta_path, "fasta")], columns=['Protein_ID', 'Sequence'])
terms_data = pd.read_csv(terms_path, sep='\t', header=None, names=['Protein_ID', 'GO_Term_ID', 'Ontology'], skiprows=1)
taxonomy_data = pd.read_csv(taxonomy_path, sep='\t', header=None, names=['Protein_ID', 'Taxon_ID'], skiprows=1)

# Data Preprocessing

In [8]:
# Combine protein sequence data with GO terms
fasta_target_path = pd.DataFrame([(record.id,
                                   str(record.seq)) for record in SeqIO.parse(fasta_target_path,
                                                                                         "fasta")],
                                 columns=['Protein_ID', 'Sequence'])

df_protein = pd.merge(fasta_data, terms_data, on='Protein_ID')

In [9]:
# Select top 1500 most frequently occurring GO terms
num_of_labels = 1500
top_labels = df_protein['GO_Term_ID'].value_counts().index[:num_of_labels].to_list()

In [10]:
df_protein = df_protein.loc[df_protein['GO_Term_ID'].isin(top_labels)]
display(df_protein)

,Protein_ID,Sequence,GO_Term_ID,Ontology
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,GO:0008152,BPO
2,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,GO:0044249,BPO
3,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,GO:0006259,BPO
4,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,GO:0009059,BPO
5,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,GO:0009987,BPO
...,...,...,...,...
5363857,A0A8I6GHU0,HCISSLKLTAFFKRSFLLSPEKHLVLLRDGRTLIGFLRSIDQFANL...,GO:0003723,MFO
5363858,A0A8I6GHU0,HCISSLKLTAFFKRSFLLSPEKHLVLLRDGRTLIGFLRSIDQFANL...,GO:0097159,MFO
5363859,A0A8I6GHU0,HCISSLKLTAFFKRSFLLSPEKHLVLLRDGRTLIGFLRSIDQFANL...,GO:1901363,MFO
5363860,A0A8I6GHU0,HCISSLKLTAFFKRSFLLSPEKHLVLLRDGRTLIGFLRSIDQFANL...,GO:0003674,MFO


In [11]:
# Ensure 'GO_Term_ID' column is in list format
df_protein['GO_Term_ID'] = df_protein['GO_Term_ID'].apply(lambda x: [x])

In [12]:
# Group by Protein_ID and aggregate GO_Term_ID lists
df_protein = df_protein.groupby('Protein_ID').agg({
    'Sequence': 'first',
    'GO_Term_ID': 'sum'
}).reset_index()

In [13]:
# Encoding labels with MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=top_labels)
df_protein = df_protein.join(pd.DataFrame(mlb.fit_transform(df_protein.pop('GO_Term_ID')),
                                          columns=mlb.classes_,
                                          index=df_protein.index))

In [14]:
print("Unique labels after MultiLabelBinarizer:", len(mlb.classes_))

Unique labels after MultiLabelBinarizer: 1500


In [15]:
display(df_protein.head())

,Protein_ID,Sequence,GO:0005575,GO:0008150,GO:0110165,GO:0003674,GO:0005622,GO:0009987,GO:0043226,GO:0043229,...,GO:0031345,GO:0034250,GO:0140053,GO:0098802,GO:0045861,GO:0051783,GO:0001818,GO:0031674,GO:0051048,GO:0070828
0,A0A009IHW8,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A0A021WW32,MFYEHIILAKKGPLARIWLAAHWDKKITKAHVFETNIEKSVEGILQ...,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,A0A021WZA4,MKYINCTQPAIDDFPRDLFSEAQRQSGAVVLHVIASLYLFVALAVV...,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A0A023FBW4,MTSHGAVKIAIFAVIALHSIFECLSKPQILQRTDHSTDSDWDPQMC...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A0A023FBW7,MKVLLYIAASCLMLLALNVSAENTQQEEEDYDYGTDTCPFPVLANK...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
def encoding(df):
    data = df['GO_Term_ID'].values

    # get labels programatically from your data
    labels = []
    for nested_list in data:
        for label in nested_list:
            if label not in labels:
                labels.append(label)

    # get labels programatically from your data
    labels = []
    for nested_list in data:
        for label in nested_list:
            if label not in labels:
                labels.append(label)

    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    labels_dict = {}
    for i in range(len(labels)):
        labels_dict[labels[i]] = encoded_labels[i]

    encoded_data = []
    for labels_list in data:
        # for each label in a nested list replace it with the encoded value from dict
        encoded_data_list = [l.replace(l, str(labels_dict[l])) for l in labels_list]
        encoded_data.append(encoded_data_list)

    df['GO_Term_ID'] = encoded_data

    return df

In [17]:
print("Unique labels after MultiLabelBinarizer:", len(mlb.classes_))

Unique labels after MultiLabelBinarizer: 1500


In [18]:
# Prepare data for modeling
X = df_protein['Sequence'].values
Y = df_protein[df_protein.columns[~df_protein.columns.isin(['Protein_ID',
                                                           'Sequence',
                                                           'Ontology'])]].values

Y = Y.astype('float32')

In [20]:
# Tokenize sequences
max_chars = len(set([*''.join(X)]))
# max_len = int(np.mean([len(i) for i in df_protein['Sequence']]))
max_len = 1200
tok = Tokenizer(num_words=max_chars, char_level=True)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [21]:
sequences_matrix.shape

(142246, 1024)

In [22]:
Y.shape

(142246, 1500)

In [24]:
# Define custom metrics
def precision(y_true, y_pred):
    y_pred_binary = K.cast(K.greater(y_pred, 0.5), dtype='float32')
    true_positives = K.sum(K.cast(K.equal(y_true * y_pred_binary, 1), dtype='float32'))
    predicted_positives = K.sum(y_pred_binary)
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    y_pred_binary = K.cast(K.greater(y_pred, 0.5), dtype='float32')
    true_positives = K.sum(K.cast(K.equal(y_true * y_pred_binary, 1), dtype='float32'))
    actual_positives = K.sum(y_true)
    recall = true_positives / (actual_positives + K.epsilon())
    return recall

# Build Model and Compile Model

In [25]:
# Build LSTM model
embedding_vector_features = 64

model=Sequential()
model.add(Input(shape=(max_len,)))
model.add(Embedding(max_chars, embedding_vector_features))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(units=64))
model.add(Dropout(rate=0.3))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense((num_of_labels), activation='sigmoid'))

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy', tf.keras.metrics.AUC(name='auc'), precision, recall])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1024, 64)          1600      
                                                                 
 lstm (LSTM)                 (None, 1024, 128)         98816     
                                                                 
 dropout (Dropout)           (None, 1024, 128)         0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

In [ ]:
# Train the model
X_train, X_val, y_train, y_val = train_test_split(sequences_matrix, Y, test_size=0.2)
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_val, y_val))

In [ ]:
# Save the trained model and training history
model.save('/content/drive/MyDrive/Eksperimen_CAFA5/Result/model_128_64.h5')

In [ ]:
# Save training history to CSV
history_df = pd.DataFrame(history.history)
history_df.to_csv('/content/drive/MyDrive/Eksperimen_CAFA5/Result/training_128_64_64.csv', index=False)

# Evaluasi model

In [ ]:
# Evaluate model
y_pred = model.predict(sequences_matrix)

In [ ]:
# Visualize training metrics
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(25, 5))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].title.set_text('model loss')
axes[0].set(xlabel='epoch', ylabel='loss')
axes[0].legend(['train loss', 'valid loss'], loc='upper right')

axes[1].plot(history.history['binary_accuracy'])
axes[1].plot(history.history['val_binary_accuracy'])
axes[1].title.set_text('model accuracy')
axes[1].set(xlabel='epoch', ylabel='accuracy')
axes[1].legend(['train acc', 'valid acc'], loc='upper right')

axes[2].plot(history.history['auc'])
axes[2].plot(history.history['val_auc'])
axes[2].title.set_text('model auc')
axes[2].set(xlabel='epoch', ylabel='auc')
axes[2].legend(['train auc', 'valid auc'], loc='upper right')

axes[3].plot(history.history['precision'])
axes[3].plot(history.history['val_precision'])
axes[3].title.set_text('model precision')
axes[3].set(xlabel='epoch', ylabel='precision')
axes[3].legend(['train prec', 'valid prec'], loc='upper right')

axes[4].plot(history.history['recall'])
axes[4].plot(history.history['val_recall'])
axes[4].title.set_text('model recall')
axes[4].set(xlabel='epoch', ylabel='recall')
axes[4].legend(['train rec', 'valid rec'], loc='upper right')

# fig.tight_layout()
fig.savefig('/content/drive/MyDrive/TA/Result/eksperimen_128_64.png') # save the figure to file

# Prediksi

In [ ]:
# Load test fasta data
test_fasta_path = "/content/drive/MyDrive/TA/Test (Targets)/testsuperset.fasta"
test_fasta_data = pd.DataFrame([(record.id, str(record.seq)) for record in SeqIO.parse(test_fasta_path, "fasta")], columns=['Protein_ID', 'Sequence'])

# Preprocess test data
X_test = test_fasta_data['Sequence'].values
sequences_test = tok.texts_to_sequences(X_test)
sequences_test_matrix = sequence.pad_sequences(sequences_test, maxlen=max_len)

# Load trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/TA/Result/model_128_64.h5', custom_objects={'precision': precision, 'recall': recall})

# Predict
Y_pred = model.predict(sequences_test_matrix)

# Create a dictionary to map GO_Term_ID to Ontology
ontology_dict = terms_data.set_index('GO_Term_ID')['Ontology'].to_dict()

# Save predictions
np.savetxt('/content/drive/MyDrive/TA/Result/predictionseksperimen12_update.csv', Y_pred, delimiter=',')

# Visualize predictions with GO terms and ontology
predicted_labels = mlb.inverse_transform(Y_pred > 0.5)

num_examples = 5
for i in range(num_examples):
    print(f"Example {i + 1}:")
    print(f"Actual Sequence: {X_test[i]}")
    print("Predicted GO Terms with Probabilities and Ontology:")
    for go_term, prob in zip(mlb.classes_, Y_pred[i]):
        if prob > 0.5:  # Threshold untuk menampilkan label prediksi
            ontology = ontology_dict.get(go_term, 'Unknown')
            print(f"{go_term} ({ontology}): {prob:.4f}")
    print()

# Save the predictions with GO terms, probabilities, and ontology
with open('/content/drive/MyDrive/TA/Result/predictionseksperimen1_detailed1_update.csv', 'w') as f:
    f.write("Protein_ID,GO_Term_ID,Prediction_Score,Sequence,Ontology\n")
    for i in range(len(X_test)):
        for go_term, prob in zip(mlb.classes_, Y_pred[i]):
            if prob > 0.5:
                ontology = ontology_dict.get(go_term, 'Unknown')
                f.write(f"{test_fasta_data['Protein_ID'][i]},{go_term},{prob:.4f},{X_test[i]},{ontology}\n")

4434/4434 [==============================] - 106s 24ms/step
Example 1:
Actual Sequence: MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSSWRVISSIEQKTERNEKKQQMGKEYREKIEAELQDICNDVLELLDKYLILNATQAESKVFYLKMKGDYFRYLSEVASGENKQTTVSNSQQAYQEAFEISKKEMQPTHPIRLGLALNFSVFYYEILNSPEKACSLAKTAFDEAIAELDTLNEESYKDSTLIMQLLRDNLTLWTSENQGDEGDAGEGEN
Predicted GO Terms with Probabilities and Ontology:
GO:0005575 (CCO): 0.7147
GO:0008150 (BPO): 0.7945
GO:0110165 (CCO): 0.7012
GO:0003674 (MFO): 0.6419
GO:0005622 (CCO): 0.6175
GO:0009987 (BPO): 0.6123
GO:0043226 (CCO): 0.5439
GO:0043229 (CCO): 0.5215
GO:0005488 (MFO): 0.5717
GO:0005515 (MFO): 0.5311
GO:0065007 (BPO): 0.5885
GO:0050789 (BPO): 0.5757
GO:0050794 (BPO): 0.5364

Example 2:
Actual Sequence: MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLSVAYKNVIGARRASWRIISSIEQKEENKGGEDKLKMIREYRQMVETELKLICCDILDVLDKHLIPAANTGESKVFYYKMKGDYHRYLAEFATGNDRKEAAENSLVAYKAASDIAMTELPPTHPIRLGLALNFSVFYYEILNSPDRACRLAKAAFDDAIAELDTLSEESYKDSTLIMQLLRDNLTLWTSDMQGDGEEQNKEALQDVEDENQ
Pre